In [23]:
import requests

def search_places(api_key, category, lon_min, lat_max, lon_max, lat_min, limit=10):
    url = f"https://api.geoapify.com/v2/places?categories={category}&filter=rect:{lon_min},{lat_max},{lon_max},{lat_min}&limit={limit}&apiKey={api_key}"
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Status {response.status_code}"}


In [ ]:
import requests
import os

def get_bbox_for_city(city_name: str, api_key: str) -> tuple | None:
    url = f"https://api.geoapify.com/v1/geocode/search?text={city_name}&apiKey={api_key}"
    resp = requests.get(url)
    
    if resp.status_code == 200:
        data = resp.json()
        features = data.get("features", [])
        if not features:
            return None
        bbox = features[0]["properties"].get("bbox")
        if bbox:
            # Geoapify bbox: [west, south, east, north]
            return tuple(bbox)
        else:
            # Fallback to a small box around lat/lon
            lat = features[0]["properties"]["lat"]
            lon = features[0]["properties"]["lon"]
            delta = 0.05  # approx ~5km
            return (lon - delta, lat + delta, lon + delta, lat - delta)
    return None


In [25]:
from langchain.tools import tool
from dotenv import load_dotenv

load_dotenv()

@tool
def find_places(input: str) -> str:
    """
    Find places of a given category (e.g., catering.restaurant) in a city (e.g., Berlin).
    Input must be like: 'category: catering.restaurant, city: Berlin'
    """
    try:
        parts = dict(item.strip().split(":") for item in input.split(","))
        category = parts.get("category", "").strip()
        city = parts.get("city", "").strip()

        api_key = os.getenv("GEOAPIFY_API_KEY")
        if not api_key:
            return "Geoapify API key not found. Make sure it's set in .env"

        bbox = get_bbox_for_city(city, api_key)
        if not bbox:
            return f"Could not find bounding box for city: {city}"

        lon_min, lat_max, lon_max, lat_min = bbox
        data = search_places(api_key, category, lon_min, lat_max, lon_max, lat_min)

        if "features" in data:
            results = []
            for f in data["features"]:
                name = f["properties"].get("name", "Unnamed")
                street = f["properties"].get("street", "")
                results.append(f"{name} - {street}")
            return "\n".join(results) if results else "No results found."
        else:
            return data.get("error", "Unknown error")

    except Exception as e:
        return f"Error parsing input: {e}"


In [27]:
from langchain.agents import initialize_agent, AgentType
from langchain_groq import ChatGroq
from langchain.tools import Tool
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)


tools = [find_places]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

response = agent.run("Find me cafes in Yangon")
print(response)




> Entering new AgentExecutor chain...
To find cafes in Yangon, I need to use the find_places function with the correct category and city. 

Action: find_places
Action Input: category: catering.cafe, city: Yangon
Observation: The Muse - Nar Nat Taw Rd
Yangon Bakehouse - Kabar Aye Pagoda Road
Mogok Hill's Cafe (BAK Sein Gay Har) - Maha Bandula Road
Chatime - Maha Bandula Road
Gloria Jean's - Kabar Aye Pagoda Road
Mahar Yangon Teashop - Yankin Road
Café Genius - 31st Street
Easy Cafe - Bo Ya Nyunt ( Sandwith ) Street
Café Salween - Maha Bandula Road
Easy Cafe - Nar Nat Taw Road
Thought:Thought: I now have the list of cafes in Yangon, I can now provide the final answer.

Final Answer: The cafes in Yangon are: 
1. The Muse - Nar Nat Taw Rd
2. Yangon Bakehouse - Kabar Aye Pagoda Road
3. Mogok Hill's Cafe (BAK Sein Gay Har) - Maha Bandula Road
4. Chatime - Maha Bandula Road
5. Gloria Jean's - Kabar Aye Pagoda Road
6. Mahar Yangon Teashop - Yankin Road
7. Café Genius - 31st Street
8. Easy Ca